# Bins and Buckets

In [1]:
buckets = [2,3,5,2,1,4,5]
bins = [5,6,8,4,7]

In [2]:
print(f'Total water: {sum(buckets)}')
print(f'Total bin capacity: {sum(bins)}')

Total water: 22
Total bin capacity: 30


**We can see that the allocation should be as follows:**
* Bucket 0: 2 units -> Bin 0
* Bucket 1: 3 units -> Bin 0
* Bucket 2: 5 units -> Bin 1
* Bucket 3: 1 unit &nbsp;-> Bin 1
* Bucket 3: 1 unit &nbsp;-> Bin 2
* Bucket 4: 1 unit &nbsp;-> Bin 2
* Bucket 5: 4 units -> Bin 2
* Bucket 6: 2 units -> Bin 2
* Bucket 6: 3 units -> Bin 3

At which point we will have exhausted all the water in our buckets.

## Bins and Buckets with Python

In [3]:
bucket_allocation = list() # tuples (bucket_index, bin_index, amount of allocation)

In [4]:
exit_flag = False
# Set up iterator for buckets
current_bucket_idx = 0
# Set up dynamic tracker for bucket contents
buckets_tracker = buckets.copy()

# Iterate through each bin as main unit
for i, bin_i in enumerate(bins):
    
    if exit_flag:
        break
    
    # While the current bin still has capacity for more water
    while bin_i > 0:
        
        # If the current bucket under review has less (or equal) water than the current capacity in the bin
        if buckets_tracker[current_bucket_idx] <= bin_i:
            # 1. Update the allocation so that the bucket empties all its contents into the bin
            bucket_allocation.append((current_bucket_idx, i, buckets_tracker[current_bucket_idx]))
            # 2. Update the capacity of the bin to account for the water added
            bin_i = bin_i - buckets_tracker[current_bucket_idx]
            # 3. Update the bucket tracker to note there is no more water in the bucket
            buckets_tracker[current_bucket_idx] = 0
            # 4. Iterate the bucket tracker to move onto the next bucket
            current_bucket_idx += 1
            
            # Exit if we run out of buckets
            if current_bucket_idx >= len(buckets_tracker):
                exit_flag = True
                break
        
        else:
        # If the current bucket has more water than the capacity of the current bin
            # 1. Update the allocation so that the bucket empties only to the current remaining capacity of the bin
            bucket_allocation.append((current_bucket_idx, i, bin_i))
            # 2. Update the bucket tracker to record the removed water added to the current bin
            buckets_tracker[current_bucket_idx] = buckets_tracker[current_bucket_idx] - bin_i
            # 3. Update the bin capacity to be zero so we can move onto the next bin
            bin_i = 0

In [5]:
buckets_tracker # We allocate all the water

[0, 0, 0, 0, 0, 0, 0]

In [6]:
bucket_allocation # Let's show this more clearly in a pandas dataframe below

[(0, 0, 2),
 (1, 0, 3),
 (2, 1, 5),
 (3, 1, 1),
 (3, 2, 1),
 (4, 2, 1),
 (5, 2, 4),
 (6, 2, 2),
 (6, 3, 3)]

In [7]:
import pandas as pd
import numpy as np

In [54]:
pd.__version__

'1.4.3'

In [55]:
np.__version__

'1.21.5'

In [8]:
pd.DataFrame.from_records(bucket_allocation, columns=['bucket_ind','bin_ind','amount'])

,bucket_ind,bin_ind,amount
0,0,0,2
1,1,0,3
2,2,1,5
3,3,1,1
4,3,2,1
5,4,2,1
6,5,2,4
7,6,2,2
8,6,3,3


However, we can even extend this to include an excess of water.

In [9]:
buckets.extend([10])
buckets

[2, 3, 5, 2, 1, 4, 5, 10]

In [10]:
print(f'Total water: {sum(buckets)}')
print(f'Total bin capacity: {sum(bins)}')

Total water: 32
Total bin capacity: 30


In [11]:
bucket_allocation = list()
exit_flag = False
current_bucket_idx = 0
buckets_tracker = buckets.copy()

for i, bin_i in enumerate(bins):
    
    if exit_flag:
        break
    
    while bin_i > 0:
        
        if buckets_tracker[current_bucket_idx] <= bin_i:
            bucket_allocation.append((current_bucket_idx, i, buckets_tracker[current_bucket_idx]))
            bin_i = bin_i - buckets_tracker[current_bucket_idx]
            buckets_tracker[current_bucket_idx] = 0
            current_bucket_idx += 1
            
            if current_bucket_idx >= len(buckets_tracker):
                exit_flag = True
                break
        
        else:
            bucket_allocation.append((current_bucket_idx, i, bin_i))
            buckets_tracker[current_bucket_idx] = buckets_tracker[current_bucket_idx] - bin_i
            bin_i = 0
            
bucket_allocation.append(('Excess Water', np.NaN, sum(buckets_tracker)))

In [12]:
python_allocation = pd.DataFrame.from_records(bucket_allocation, columns=['bucket_ind','bin_ind','water_allocation'])
python_allocation

,bucket_ind,bin_ind,water_allocation
0,0,0.0,2
1,1,0.0,3
2,2,1.0,5
3,3,1.0,1
4,3,2.0,1
5,4,2.0,1
6,5,2.0,4
7,6,2.0,2
8,6,3.0,3
9,7,3.0,1


## Bins and Buckets with SQL

In [13]:
import sqlite3
# Create a database on the fly in memory
conn = sqlite3.connect(':memory:')
cursor = conn.cursor()

SQL makes use of tables so we will create them using pandas.

In [14]:
buckets_tbl = pd.DataFrame({'bucket_ind':range(len(buckets)), 'water':buckets})
display(buckets_tbl)

buckets_tbl.to_sql('buckets', con=conn, index=False)

,bucket_ind,water
0,0,2
1,1,3
2,2,5
3,3,2
4,4,1
5,5,4
6,6,5
7,7,10


8

In [15]:
bins_tbl = pd.DataFrame({'bin_ind':range(len(bins)), 'capacity':bins})
display(bins_tbl)

bins_tbl.to_sql('bins', con=conn, index=False)

,bin_ind,capacity
0,0,5
1,1,6
2,2,8
3,3,4
4,4,7


5

For both tables we want to keep track of the running cumulative sum of water for buckets and capacity for bins.

In [16]:
query = """
SELECT
    bucket_ind,
    water,
    SUM(water) OVER (ORDER BY bucket_ind ASC ROWS BETWEEN UNBOUNDED PRECEDING AND CURRENT ROW) AS water_cum,
    COALESCE(SUM(water) OVER (
        ORDER BY bucket_ind ASC ROWS BETWEEN UNBOUNDED PRECEDING AND 1 PRECEDING
    ),0) AS prev_water_cum
FROM buckets
"""

query_data = pd.read_sql(
    query,
    con=conn
)

query_data

,bucket_ind,water,water_cum,prev_water_cum
0,0,2,2,0
1,1,3,5,2
2,2,5,10,5
3,3,2,12,10
4,4,1,13,12
5,5,4,17,13
6,6,5,22,17
7,7,10,32,22


In [17]:
query = """
SELECT
    bin_ind,
    capacity,
    SUM(capacity) OVER (
        ORDER BY bin_ind ASC ROWS BETWEEN UNBOUNDED PRECEDING and CURRENT ROW
    ) AS capacity_cum,
    COALESCE(SUM(capacity) OVER (
        ORDER BY bin_ind ASC ROWS BETWEEN UNBOUNDED PRECEDING and 1 PRECEDING
    ), 0) AS prev_capacity_cum
FROM
    bins
"""

query_data = pd.read_sql(
    query,
    con=conn
)

query_data

,bin_ind,capacity,capacity_cum,prev_capacity_cum
0,0,5,5,0
1,1,6,11,5
2,2,8,19,11
3,3,4,23,19
4,4,7,30,23


We next make use of a Cross Join to join every bucket with every bin in order to make the connection between the buckets and the bins. We have to do this in a SQL context which is one of the main ways this differs from a Python context where we can make these connections sequentially. In other words, we can fill a bin with a bucket and then determine whether the bucket is exhausted with that bin or if it can be assigned to another bin; in SQL we start by hypothetically assigning every bucket to every bin.

In [18]:
query = """
WITH buckets_data AS (
    SELECT
        bucket_ind,
        water,
        SUM(water) OVER (ORDER BY bucket_ind ASC ROWS BETWEEN UNBOUNDED PRECEDING AND CURRENT ROW) AS water_cum,
        COALESCE(SUM(water) OVER (
            ORDER BY bucket_ind ASC ROWS BETWEEN UNBOUNDED PRECEDING AND 1 PRECEDING
        ),0) AS prev_water_cum
    FROM buckets
), bins_data AS (
    SELECT
        bin_ind,
        capacity,
        SUM(capacity) OVER (
            ORDER BY bin_ind ASC ROWS BETWEEN UNBOUNDED PRECEDING and CURRENT ROW
        ) AS capacity_cum,
        COALESCE(SUM(capacity) OVER (
            ORDER BY bin_ind ASC ROWS BETWEEN UNBOUNDED PRECEDING and 1 PRECEDING
        ), 0) AS prev_capacity_cum
    FROM
        bins
)
SELECT
    *
FROM
    buckets_data
    CROSS JOIN bins_data
"""

query_data = pd.read_sql(
    query,
    con=conn
)

query_data

,bucket_ind,water,water_cum,prev_water_cum,bin_ind,capacity,capacity_cum,prev_capacity_cum
0,0,2,2,0,0,5,5,0
1,0,2,2,0,1,6,11,5
2,0,2,2,0,2,8,19,11
3,0,2,2,0,3,4,23,19
4,0,2,2,0,4,7,30,23
5,1,3,5,2,0,5,5,0
6,1,3,5,2,1,6,11,5
7,1,3,5,2,2,8,19,11
8,1,3,5,2,3,4,23,19
9,1,3,5,2,4,7,30,23


We need to return only the rows with the appropriate match between bucket index and bin index.

buckets_data and bins_data is the same as the two queries we ran above to show the cumulative water and cumulative capacity. 

We filter the crossjoin based on the availability of water to assign to open capacity. This is done by the where statement that selects rows where the previous amount of cumulative water up to that row is less than the capacity of that row (so we know there is still capacity available) and where the cumulative water up to that row is greater than the previous cumulative capacity up to that row (so we know that the total water is greater than the previous capacity and we have water left to assign to the capacity in that row). 

In [19]:
query = """
WITH buckets_data AS (
    SELECT
        bucket_ind,
        water,
        SUM(water) OVER (ORDER BY bucket_ind ASC ROWS BETWEEN UNBOUNDED PRECEDING AND CURRENT ROW) AS water_cum,
        COALESCE(SUM(water) OVER (
            ORDER BY bucket_ind ASC ROWS BETWEEN UNBOUNDED PRECEDING AND 1 PRECEDING
        ),0) AS prev_water_cum
    FROM buckets
), bins_data AS (
    SELECT
        bin_ind,
        capacity,
        SUM(capacity) OVER (
            ORDER BY bin_ind ASC ROWS BETWEEN UNBOUNDED PRECEDING and CURRENT ROW
        ) AS capacity_cum,
        COALESCE(SUM(capacity) OVER (
            ORDER BY bin_ind ASC ROWS BETWEEN UNBOUNDED PRECEDING and 1 PRECEDING
        ), 0) AS prev_capacity_cum
    FROM
        bins
)
SELECT
    bucket_ind,
    bin_ind,
    water,
    capacity,
    water_cum,
    capacity_cum,
    prev_water_cum,
    prev_capacity_cum
FROM
    buckets_data
    CROSS JOIN bins_data
WHERE
    prev_water_cum < capacity_cum
    AND water_cum > prev_capacity_cum
"""

query_data = pd.read_sql(
    query,
    con=conn
)

query_data

,bucket_ind,bin_ind,water,capacity,water_cum,capacity_cum,prev_water_cum,prev_capacity_cum
0,0,0,2,5,2,5,0,0
1,1,0,3,5,5,5,2,0
2,2,1,5,6,10,11,5,5
3,3,1,2,6,12,11,10,5
4,3,2,2,8,12,19,10,11
5,4,2,1,8,13,19,12,11
6,5,2,4,8,17,19,13,11
7,6,2,5,8,22,19,17,11
8,6,3,5,4,22,23,17,19
9,7,3,10,4,32,23,22,19


Now we need to calculate the size of the allocation between each bucket and the corresponding bin.

In [20]:
query = """
WITH buckets_data AS (
    SELECT
        bucket_ind,
        water,
        SUM(water) OVER (ORDER BY bucket_ind ASC ROWS BETWEEN UNBOUNDED PRECEDING AND CURRENT ROW) AS water_cum,
        COALESCE(SUM(water) OVER (
            ORDER BY bucket_ind ASC ROWS BETWEEN UNBOUNDED PRECEDING AND 1 PRECEDING
        ),0) AS prev_water_cum
    FROM buckets
), bins_data AS (
    SELECT
        bin_ind,
        capacity,
        SUM(capacity) OVER (
            ORDER BY bin_ind ASC ROWS BETWEEN UNBOUNDED PRECEDING and CURRENT ROW
        ) AS capacity_cum,
        COALESCE(SUM(capacity) OVER (
            ORDER BY bin_ind ASC ROWS BETWEEN UNBOUNDED PRECEDING and 1 PRECEDING
        ), 0) AS prev_capacity_cum
    FROM
        bins
)
SELECT
    *,
    CASE
        WHEN current_balance >= 0 THEN IIF(updated_capacity >= water, water, updated_capacity)
        WHEN current_balance < 0 THEN IIF(capacity >= updated_water, updated_water, capacity)
        ELSE 0
    END AS water_allocation
FROM (
    SELECT
        bucket_ind,
        bin_ind,
        water,
        capacity,
        water_cum,
        capacity_cum,
        prev_water_cum,
        prev_capacity_cum,
        prev_water_cum - prev_capacity_cum AS current_balance,
        IIF(prev_water_cum - prev_capacity_cum >= 0, capacity - (prev_water_cum - prev_capacity_cum), NULL) AS updated_capacity,
        IIF(prev_water_cum - prev_capacity_cum < 0, water + (prev_water_cum - prev_capacity_cum), NULL) AS updated_water
    FROM
        buckets_data
        CROSS JOIN bins_data
    WHERE
        prev_water_cum < capacity_cum
        AND water_cum > prev_capacity_cum
) data
UNION
SELECT
    'Excess Water' AS bucket_ind,
    NULL AS bin_ind,
    NULL AS water,
    NULL AS capacity,
    NULL AS water_cum,
    NULL AS capacity_cum,
    NULL AS prev_water_cum,
    NULL AS prev_capacity_cum,
    NULL AS current_balance,
    NULL AS updated_capacity,
    NULL AS updated_water,
    IIF(
        (SELECT MAX(water_cum) FROM buckets_data) > (SELECT MAX(capacity_cum) FROM bins_data),
        (SELECT MAX(water_cum) FROM buckets_data) - (SELECT MAX(capacity_cum) FROM bins_data),
        0
    ) AS water_allocation
"""

sql_allocation = pd.read_sql(
    query,
    con=conn
)

sql_allocation

,bucket_ind,bin_ind,water,capacity,water_cum,capacity_cum,prev_water_cum,prev_capacity_cum,current_balance,updated_capacity,updated_water,water_allocation
0,0,0.0,2.0,5.0,2.0,5.0,0.0,0.0,0.0,5.0,NaN,2
1,1,0.0,3.0,5.0,5.0,5.0,2.0,0.0,2.0,3.0,NaN,3
2,2,1.0,5.0,6.0,10.0,11.0,5.0,5.0,0.0,6.0,NaN,5
3,3,1.0,2.0,6.0,12.0,11.0,10.0,5.0,5.0,1.0,NaN,1
4,3,2.0,2.0,8.0,12.0,19.0,10.0,11.0,-1.0,NaN,1.0,1
5,4,2.0,1.0,8.0,13.0,19.0,12.0,11.0,1.0,7.0,NaN,1
6,5,2.0,4.0,8.0,17.0,19.0,13.0,11.0,2.0,6.0,NaN,4
7,6,2.0,5.0,8.0,22.0,19.0,17.0,11.0,6.0,2.0,NaN,2
8,6,3.0,5.0,4.0,22.0,23.0,17.0,19.0,-2.0,NaN,3.0,3
9,7,3.0,10.0,4.0,32.0,23.0,22.0,19.0,3.0,1.0,NaN,1


In [21]:
# Check if Python and SQL return the same outputs
python_allocation.equals(sql_allocation[['bucket_ind','bin_ind','water_allocation']])

True

In [22]:
# Visually inspect
python_allocation

,bucket_ind,bin_ind,water_allocation
0,0,0.0,2
1,1,0.0,3
2,2,1.0,5
3,3,1.0,1
4,3,2.0,1
5,4,2.0,1
6,5,2.0,4
7,6,2.0,2
8,6,3.0,3
9,7,3.0,1


In [23]:
# Visually inspect
sql_allocation[['bucket_ind','bin_ind','water_allocation']]

,bucket_ind,bin_ind,water_allocation
0,0,0.0,2
1,1,0.0,3
2,2,1.0,5
3,3,1.0,1
4,3,2.0,1
5,4,2.0,1
6,5,2.0,4
7,6,2.0,2
8,6,3.0,3
9,7,3.0,1


# Working with Payments Data

### Load Charge Data

In [87]:
charges = (
    pd.read_csv(
        'charges.csv',
        parse_dates=['scheduled_payment_date']
    )
)

charges

,customer_id,scheduled_payment_date,scheduled_rent,scheduled_utilities,scheduled_late_fee,scheduled_total_payment
0,1005,2021-02-10,1166,175,0,1341
1,1005,2021-02-26,1166,175,0,1341
2,1005,2021-03-10,1166,175,0,1341
3,1005,2021-03-26,1166,175,0,1341
4,1005,2021-04-10,1166,175,0,1341
...,...,...,...,...,...,...
63,1003,2021-07-17,1750,262,0,2012
64,1003,2021-08-02,1750,262,0,2012
65,1003,2021-08-17,1750,262,0,2012
66,1003,2021-09-02,1750,262,0,2012


Inspect this data for customer_id 1002 to observe what it looks like.

In [89]:
charges.query('customer_id == 1002')

,customer_id,scheduled_payment_date,scheduled_rent,scheduled_utilities,scheduled_late_fee,scheduled_total_payment
12,1002,2020-12-27,2083,0,0,2083
13,1002,2021-01-12,2083,0,0,2083
14,1002,2021-01-27,2083,0,0,2083
15,1002,2021-02-12,2083,0,0,2083
16,1002,2021-02-27,2083,0,0,2083
17,1002,2021-03-12,2083,0,0,2083
18,1002,2021-03-27,2083,0,0,2083
19,1002,2021-04-12,2083,0,0,2083
20,1002,2021-04-27,2083,0,0,2083
21,1002,2021-05-12,2083,0,0,2083


The charges are recorded as "scheduled" payments with columns for rent, utilities, and late fees. For this discussion we are only interested in the total charge so we will keep only the scheduled payment date, the customer_id, and the scheduled total payment.

In [90]:
charges = charges.filter(['customer_id','scheduled_payment_date','scheduled_total_payment'])
charges

,customer_id,scheduled_payment_date,scheduled_total_payment
0,1005,2021-02-10,1341
1,1005,2021-02-26,1341
2,1005,2021-03-10,1341
3,1005,2021-03-26,1341
4,1005,2021-04-10,1341
...,...,...,...
63,1003,2021-07-17,2012
64,1003,2021-08-02,2012
65,1003,2021-08-17,2012
66,1003,2021-09-02,2012


### Load Payment Data

In [91]:
from math import floor

In [95]:
payments = (
    pd.read_csv(
        'payments.csv',
        parse_dates=['payment_date'],
        dayfirst=True,
        infer_datetime_format=True
    )
    .sort_values(by=['customer_id','payment_date','payment_status'], ascending=[True,True,True])
    .reset_index(drop=True)
)

payments

,customer_id,payment_status,payment_date,rent_payment,utilities_payment,late_fee_payment,total_payment
0,1001,PAID,2021-03-28,0,200,0,200
1,1001,PAID,2021-04-03,1666,500,0,2166
2,1001,PAID,2021-04-19,0,200,0,200
3,1001,PAID,2021-04-23,1666,500,0,2166
4,1001,PAID,2021-05-08,0,200,0,200
...,...,...,...,...,...,...,...
65,1006,PAID,2021-05-11,875,125,0,1000
66,1006,PAID,2021-05-29,875,175,0,1050
67,1006,PAID,2021-06-15,572,125,800,1497
68,1006,RETURNED,2021-06-15,572,125,800,1497


Similar to the situation with the charge data, we are only interested the total payment so we keep only this column. However, there are some additional cleaning steps for the payment data. The payment data includes records that are both "paid" and "returned" so we need to remove the payments that were returned.

Take a look at the payment records for customer 1002. We can see that the RETURNED payments do not necessarily correspond to the same day and in fact payments may be reverted over a month later by observing indexes 23 and 30.

In [99]:
(
    payments
    .query('customer_id == 1002')
)

,customer_id,payment_status,payment_date,rent_payment,utilities_payment,late_fee_payment,total_payment
16,1002,PAID,2020-12-28,2083,0,0,2083
17,1002,PAID,2021-01-13,2083,0,0,2083
18,1002,PAID,2021-01-27,2083,0,0,2083
19,1002,PAID,2021-02-12,2083,0,0,2083
20,1002,PAID,2021-02-27,2083,0,0,2083
21,1002,PAID,2021-03-12,2083,0,0,2083
22,1002,PAID,2021-03-27,2083,0,0,2083
23,1002,PAID,2021-04-27,1179,0,800,1979
24,1002,PAID,2021-05-21,2931,0,1006,3937
25,1002,PAID,2021-05-27,1385,0,593,1978


To remove returned payments we identify the returned entries and for each returned entry we search earlier payments starting from payments closest to the returned entry and working our way earlier. When a matching entry is found we remove it from the dataframe. One failing here is that if the matched payment is not the correct return, but rather an even earlier payment is the correct one we will not be able to determine this with only the data we have here.

In [103]:
def remove_returned_payments(df):
    
    reversions = df.loc[df['payment_status'] == 'RETURNED']
    
    payments = df.loc[df['payment_status'] != 'RETURNED']
    
    indexes_to_remove = reversions.index.tolist()
    
    for reversion in reversions.itertuples():
        
        for payment in payments.loc[payments.index < reversion.Index][::-1].itertuples():

            if all([
                reversion.rent_payment == payment.rent_payment,
                reversion.utilities_payment == payment.utilities_payment,
                reversion.late_fee_payment == payment.late_fee_payment,
            ]):
                
                indexes_to_remove.append(payment.Index)
                break
    
    return df.drop(index=indexes_to_remove)
    

In [104]:
# An example of handling reverted payments for customer_id 1002
(
    payments
    .query('customer_id == 1002')
    .sort_values(
        ['customer_id','payment_date','payment_status'], 
        ascending=[True, True, True]
    )
    .groupby('customer_id', group_keys=False)
    .apply(remove_returned_payments)
)

,customer_id,payment_status,payment_date,rent_payment,utilities_payment,late_fee_payment,total_payment
16,1002,PAID,2020-12-28,2083,0,0,2083
17,1002,PAID,2021-01-13,2083,0,0,2083
18,1002,PAID,2021-01-27,2083,0,0,2083
19,1002,PAID,2021-02-12,2083,0,0,2083
20,1002,PAID,2021-02-27,2083,0,0,2083
21,1002,PAID,2021-03-12,2083,0,0,2083
22,1002,PAID,2021-03-27,2083,0,0,2083
26,1002,PAID,2021-06-02,2083,0,0,2083
27,1002,PAID,2021-06-02,2083,0,0,2083
28,1002,PAID,2021-06-02,4167,0,0,4167


In [111]:
payments = (
    pd.read_csv(
        'payments.csv',
        parse_dates=['payment_date'],
        dayfirst=True,
        infer_datetime_format=True
    )
    .sort_values(by=['customer_id','payment_date','payment_status'], ascending=[True,True,True])
    # Remove returned payments by loan_number grouping
    .groupby('customer_id', group_keys=False)
    .apply(remove_returned_payments)
    .reset_index(drop=True)
)

payments

,customer_id,payment_status,payment_date,rent_payment,utilities_payment,late_fee_payment,total_payment
0,1001,PAID,2021-03-28,0,200,0,200
1,1001,PAID,2021-04-03,1666,500,0,2166
2,1001,PAID,2021-04-19,0,200,0,200
3,1001,PAID,2021-04-23,1666,500,0,2166
4,1001,PAID,2021-05-29,1666,500,0,2166
5,1001,PAID,2021-06-17,0,200,0,200
6,1002,PAID,2020-12-28,2083,0,0,2083
7,1002,PAID,2021-01-13,2083,0,0,2083
8,1002,PAID,2021-01-27,2083,0,0,2083
9,1002,PAID,2021-02-12,2083,0,0,2083


In [112]:
# No more Returned Payments
payments['payment_status'].unique()

array(['PAID'], dtype=object)

In [113]:
payments = payments.filter(['customer_id','payment_date','total_payment'])
payments

,customer_id,payment_date,total_payment
0,1001,2021-03-28,200
1,1001,2021-04-03,2166
2,1001,2021-04-19,200
3,1001,2021-04-23,2166
4,1001,2021-05-29,2166
5,1001,2021-06-17,200
6,1002,2020-12-28,2083
7,1002,2021-01-13,2083
8,1002,2021-01-27,2083
9,1002,2021-02-12,2083


## Working with Payments Data in Python

We start off by adapting the code above for Bins and Buckets in Python into a function that we apply group-wise to each set of loan records.

In [114]:
def allocate_payments(charges, payments):
    
    charges = charges.copy().sort_values(['scheduled_payment_date'], ascending=[True])
    bins = charges['scheduled_total_payment'].tolist()
    charges_idx = charges[['scheduled_payment_date','scheduled_total_payment']].reset_index(drop=True)
    
    payments = payments.copy().sort_values(['payment_date'], ascending=[True])
    buckets = payments['total_payment'].tolist()
    payments_idx = payments[['customer_id','payment_date','total_payment']].reset_index(drop=True)
    
    bucket_allocation = list() # Tuple (bucket_index, bin_index, payment_allocation)
    exit_flag = False
    current_bucket_idx = 0
    buckets_tracker = buckets.copy()

    for i, bin_i in enumerate(bins):

        if exit_flag or len(buckets) == 0:
            break

        while bin_i > 0:

            if buckets_tracker[current_bucket_idx] <= bin_i:
                bucket_allocation.append((current_bucket_idx, i, buckets_tracker[current_bucket_idx]))
                bin_i = bin_i - buckets_tracker[current_bucket_idx]
                buckets_tracker[current_bucket_idx] = 0
                current_bucket_idx += 1

                if current_bucket_idx >= len(buckets_tracker):
                    exit_flag = True
                    break

            else:
                bucket_allocation.append((current_bucket_idx, i, bin_i))
                buckets_tracker[current_bucket_idx] = buckets_tracker[current_bucket_idx] - bin_i
                bin_i = 0
    
    
    df = (
        pd.DataFrame.from_records(bucket_allocation, columns=['bucket_ind','bin_ind','payment_allocation'])
        .join(charges_idx, on='bin_ind')
        .join(payments_idx, on='bucket_ind')
        .filter([
            'customer_id',
            'scheduled_payment_date',
            'payment_date',
            'scheduled_total_payment',
            'payment_allocation',
            'total_payment'
        ])
    )
    
    return df

Let's implement this using multiprocessing to run it a bit faster than on a single core as the dataframe is somewhat large.

In [115]:
from multiprocess import Pool, cpu_count

In [116]:
cpu_count()

4

In [117]:
customer_ids = charges['customer_id'].unique()

In [119]:
charges_customers_idx = [charges.loc[charges['customer_id'] == customer_id].index for customer_id in customer_ids]

In [120]:
payments_customers_idx = [payments.loc[payments['customer_id'] == customer_id].index for customer_id in customer_ids]

In [123]:
with Pool(cpu_count()) as pool:

    allocated_payments = pd.concat(
        # Process allocate_payments function on each loan using multiple cores.
        pool.starmap(
            allocate_payments, 
            zip(
                [charges.loc[customer_idx] for customer_idx in charges_customers_idx],
                [payments.loc[customer_idx] for customer_idx in payments_customers_idx]
            )
        ), axis=0
    )


allocated_payments

,customer_id,scheduled_payment_date,payment_date,scheduled_total_payment,payment_allocation,total_payment
0,1005,2021-02-10,2021-02-10,1341,1341,1341
1,1005,2021-02-26,2021-02-26,1341,1341,1341
2,1005,2021-03-10,2021-03-16,1341,1341,1341
3,1005,2021-03-26,2021-03-25,1341,1341,1341
4,1005,2021-04-10,2021-04-15,1341,1341,1341
5,1005,2021-04-26,2021-04-27,1343,0,0
6,1005,2021-04-26,2021-04-27,1343,1341,1341
7,1005,2021-04-26,2021-05-12,1343,0,0
8,1005,2021-04-26,2021-05-12,1343,2,1341
9,1005,2021-05-10,2021-05-12,1345,1339,1341


We can explore the success of this code by looking at customer 1002.

In [124]:
allocated_payments.query('customer_id == 1002')

,customer_id,scheduled_payment_date,payment_date,scheduled_total_payment,payment_allocation,total_payment
0,1002,2020-12-27,2020-12-28,2083,2083,2083
1,1002,2021-01-12,2021-01-13,2083,2083,2083
2,1002,2021-01-27,2021-01-27,2083,2083,2083
3,1002,2021-02-12,2021-02-12,2083,2083,2083
4,1002,2021-02-27,2021-02-27,2083,2083,2083
5,1002,2021-03-12,2021-03-12,2083,2083,2083
6,1002,2021-03-27,2021-03-27,2083,2083,2083
7,1002,2021-04-12,2021-06-02,2083,2083,2083
8,1002,2021-04-27,2021-06-02,2083,2083,2083
9,1002,2021-05-12,2021-06-02,2083,2083,4167


## Working with Payments Data in SQL

I'll use the existing tables that we cleaned up in Python since the purpose of this demo is to show how to work with payments data in SQL specifically for the payment-charge allocation, not generally how to clean the data in SQL as well.

In [125]:
import sqlite3
# Create a database on the fly in memory
conn = sqlite3.connect(':memory:')
cursor = conn.cursor()

In [126]:
payments.to_sql('payments', con=conn, index=False)

pd.read_sql(
    """
    SELECT
        *
    FROM
        payments
    LIMIT 10
    """,
    con=conn
)

,customer_id,payment_date,total_payment
0,1001,2021-03-28 00:00:00,200
1,1001,2021-04-03 00:00:00,2166
2,1001,2021-04-19 00:00:00,200
3,1001,2021-04-23 00:00:00,2166
4,1001,2021-05-29 00:00:00,2166
5,1001,2021-06-17 00:00:00,200
6,1002,2020-12-28 00:00:00,2083
7,1002,2021-01-13 00:00:00,2083
8,1002,2021-01-27 00:00:00,2083
9,1002,2021-02-12 00:00:00,2083


In [127]:
charges.to_sql('charges', con=conn, index=False)

pd.read_sql(
    """
    SELECT
        *
    FROM
        charges
    LIMIT 10
    """,
    con=conn
)

,customer_id,scheduled_payment_date,scheduled_total_payment
0,1005,2021-02-10 00:00:00,1341
1,1005,2021-02-26 00:00:00,1341
2,1005,2021-03-10 00:00:00,1341
3,1005,2021-03-26 00:00:00,1341
4,1005,2021-04-10 00:00:00,1341
5,1005,2021-04-26 00:00:00,1343
6,1005,2021-05-10 00:00:00,1345
7,1005,2021-05-26 00:00:00,1341
8,1005,2021-06-10 00:00:00,1341
9,1005,2021-06-26 00:00:00,1345


Note below: We add in a payment index to the payments data to account for the fact that we do not have a transaction level identifier for payments. For example, since we only have time data to the level of a day, multiple payments on the same day would be indistinguishable using a customer_id-date identifier. However, as each row in the original payments data represents a single payment we can simply assign a payment index to each row within a customer_id grouping.

In [134]:
query = """
WITH payments_data AS (
    SELECT
        *,
        SUM(total_payment) OVER (
            PARTITION BY customer_id ORDER BY payment_ind ASC ROWS BETWEEN UNBOUNDED PRECEDING AND CURRENT ROW
        ) AS total_payment_cum,
        COALESCE(SUM(total_payment) OVER (
            PARTITION BY customer_id ORDER BY payment_ind ASC ROWS BETWEEN UNBOUNDED PRECEDING AND 1 PRECEDING
        ), 0) AS prev_total_payment_cum
    FROM (
        SELECT
            customer_id,
            payment_date,
            ROW_NUMBER() OVER (PARTITION BY customer_id ORDER BY payment_date ASC) AS payment_ind,
            total_payment
        FROM
            payments
    ) AS payments_ind
), charges_data AS (
    SELECT
        customer_id,
        scheduled_payment_date,
        scheduled_total_payment,
        SUM(scheduled_total_payment) OVER (
            PARTITION BY customer_id ORDER BY scheduled_payment_date ASC ROWS BETWEEN UNBOUNDED PRECEDING AND CURRENT ROW
        ) AS scheduled_total_payment_cum,
        COALESCE(SUM(scheduled_total_payment) OVER (
            PARTITION BY customer_id ORDER BY scheduled_payment_date ASC ROWS BETWEEN UNBOUNDED PRECEDING AND 1 PRECEDING
        ), 0) AS prev_scheduled_total_payment_cum
    FROM
        charges
)
SELECT
    *,
    CASE
        WHEN current_balance >= 0 THEN IIF(updated_charges >= total_payment, total_payment, updated_charges)
        WHEN current_balance < 0 THEN IIF(scheduled_total_payment >= updated_payments, updated_payments, scheduled_total_payment)
        ELSE 0
    END AS payment_allocation
FROM (
    SELECT
        pd.customer_id,
        pd.payment_ind,
        payment_date,
        scheduled_payment_date,
        total_payment,
        scheduled_total_payment,
        total_payment_cum,
        scheduled_total_payment_cum,
        prev_total_payment_cum,
        prev_scheduled_total_payment_cum,
        prev_total_payment_cum - prev_scheduled_total_payment_cum AS current_balance,
        IIF(prev_total_payment_cum - prev_scheduled_total_payment_cum >= 0, scheduled_total_payment - (prev_total_payment_cum - prev_scheduled_total_payment_cum), NULL) AS updated_charges,
        IIF(prev_total_payment_cum - prev_scheduled_total_payment_cum < 0, total_payment + (prev_total_payment_cum - prev_scheduled_total_payment_cum), NULL) AS updated_payments
    FROM
        payments_data AS pd
        CROSS JOIN charges_data AS cd
    WHERE
        pd.customer_id = cd.customer_id
        AND prev_total_payment_cum < scheduled_total_payment_cum
        AND total_payment_cum > prev_scheduled_total_payment_cum
) data
"""

sql_payment_allocation = pd.read_sql(
    query,
    con=conn
)

sql_payment_allocation.filter([
    'customer_id',
    'scheduled_payment_date',
    'payment_ind',
    'payment_date',
    'scheduled_total_payment',
    'payment_allocation',
    'total_payment',
    'prev_total_payment_cum',
    'prev_scheduled_total_payment_cum',
    'current_balance',
    'updated_charges',
    'updated_payments',
    'total_payment_cum',
    'scheduled_total_payment_cum'
])

,customer_id,scheduled_payment_date,payment_ind,payment_date,scheduled_total_payment,payment_allocation,total_payment,prev_total_payment_cum,prev_scheduled_total_payment_cum,current_balance,updated_charges,updated_payments,total_payment_cum,scheduled_total_payment_cum
0,1001,2021-04-03 00:00:00,1,2021-03-28 00:00:00,2166,200,200,0,0,0,2166.0,NaN,200,2166
1,1001,2021-04-03 00:00:00,2,2021-04-03 00:00:00,2166,1966,2166,200,0,200,1966.0,NaN,2366,2166
2,1001,2021-04-24 00:00:00,2,2021-04-03 00:00:00,2166,200,2166,200,2166,-1966,NaN,200.0,2366,4332
3,1001,2021-04-24 00:00:00,3,2021-04-19 00:00:00,2166,200,200,2366,2166,200,1966.0,NaN,2566,4332
4,1001,2021-04-24 00:00:00,4,2021-04-23 00:00:00,2166,1766,2166,2566,2166,400,1766.0,NaN,4732,4332
5,1001,2021-05-29 00:00:00,4,2021-04-23 00:00:00,2166,400,2166,2566,4332,-1766,NaN,400.0,4732,6498
6,1001,2021-05-29 00:00:00,5,2021-05-29 00:00:00,2166,1766,2166,4732,4332,400,1766.0,NaN,6898,6498
7,1001,2021-06-29 00:00:00,5,2021-05-29 00:00:00,2166,400,2166,4732,6498,-1766,NaN,400.0,6898,8664
8,1001,2021-06-29 00:00:00,6,2021-06-17 00:00:00,2166,200,200,6898,6498,400,1766.0,NaN,7098,8664
9,1002,2020-12-27 00:00:00,1,2020-12-28 00:00:00,2083,2083,2083,0,0,0,2083.0,NaN,2083,2083


We can again examine the output of the query by focusing on customer 1002.

In [135]:
sql_payment_allocation.query('customer_id == 1002').filter([
    'customer_id',
    'scheduled_payment_date',
    'payment_ind',
    'payment_date',
    'scheduled_total_payment',
    'payment_allocation',
    'total_payment',
    'prev_total_payment_cum',
    'prev_scheduled_total_payment_cum',
    'current_balance',
    'updated_charges',
    'updated_payments',
    'total_payment_cum',
    'scheduled_total_payment_cum'
])

,customer_id,scheduled_payment_date,payment_ind,payment_date,scheduled_total_payment,payment_allocation,total_payment,prev_total_payment_cum,prev_scheduled_total_payment_cum,current_balance,updated_charges,updated_payments,total_payment_cum,scheduled_total_payment_cum
9,1002,2020-12-27 00:00:00,1,2020-12-28 00:00:00,2083,2083,2083,0,0,0,2083.0,NaN,2083,2083
10,1002,2021-01-12 00:00:00,2,2021-01-13 00:00:00,2083,2083,2083,2083,2083,0,2083.0,NaN,4166,4166
11,1002,2021-01-27 00:00:00,3,2021-01-27 00:00:00,2083,2083,2083,4166,4166,0,2083.0,NaN,6249,6249
12,1002,2021-02-12 00:00:00,4,2021-02-12 00:00:00,2083,2083,2083,6249,6249,0,2083.0,NaN,8332,8332
13,1002,2021-02-27 00:00:00,5,2021-02-27 00:00:00,2083,2083,2083,8332,8332,0,2083.0,NaN,10415,10415
14,1002,2021-03-12 00:00:00,6,2021-03-12 00:00:00,2083,2083,2083,10415,10415,0,2083.0,NaN,12498,12498
15,1002,2021-03-27 00:00:00,7,2021-03-27 00:00:00,2083,2083,2083,12498,12498,0,2083.0,NaN,14581,14581
16,1002,2021-04-12 00:00:00,8,2021-06-02 00:00:00,2083,2083,2083,14581,14581,0,2083.0,NaN,16664,16664
17,1002,2021-04-27 00:00:00,9,2021-06-02 00:00:00,2083,2083,2083,16664,16664,0,2083.0,NaN,18747,18747
18,1002,2021-05-12 00:00:00,10,2021-06-02 00:00:00,2083,2083,4167,18747,18747,0,2083.0,NaN,22914,20830
